In [ ]:
import os, sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import laps, redis, loader
from Dijkstra.Dijkstra import *

#### Setup redis client
Defines `redis_connection` to avoid explicitly having to type the IP to make it easier to change.

In [ ]:
from redis_connection import *

### Temporary Redis Data Adding
As the server isn't up yet and we're running on local machines, the next cell serves only to add the map image to the Redis database. The PNG image is stored as a byte array on the Redis database, to achieve this `io` is used.

In [ ]:
from PIL import Image, ImageDraw
import io

img = Image.open("../DeepStar/map.png", mode='r')

img_byte_array = io.BytesIO()
img.save(img_byte_array, format='PNG')
img_byte_array = img_byte_array.getvalue()

redis_connection.set(2, img_byte_array)

Setup handler function, which is the function responsible for taking in the input and using the Dijkstra module to return an output. The `Job request` API call, is in JSON format, and is formatted like this:
```JSON
{
    "job_id": int32,                                                                     
    "start": {
        "x": int32,                                                                       
        "y": int32                                                                        
    },  
    "stop": {
        "x": int32,                                                                       
        "y": int32                                                                        
    }   
}
```
The parsed JSON is passed as a Python dictionary to runner, with the help of the `laps.py`. The `Job request` API call expects the `Job result` answer in the JSON format as well, formatted as:
```JSON
{
    "job_id": int32,  
    "points": [
        "x": int32,                                                                       
        "y": int32                                                                        
    ]   
}
```
Which `points` being a list of entries with a `x` and `y` field.

In [ ]:
def handler(runner, job):
    # Get start and stop points from the job request
    start = (job["start"]["x"], job["start"]["y"])
    stop = (job["stop"]["x"], job["stop"]["y"])
    
    # Read the PNG image from the Redis key given by the request
    img_byte_array = redis_connection.get(job["map_id"])
    # Make a grid of the heightvalues from the image
    grid = []
    with Image.open(io.BytesIO(img_byte_array)) as img:
        pixels = img.load()
        for x in range(256):
            grid.append([])
            for y in range(256):
                grid[x].append(Node(pixels[x, y][0] / 255, (x, y)))

    # Use the Dijkstra module on the grid, along with start and stop
    # to calculate the path.
    edges = get_edges(grid)
    dijkstra_path = dijkstra(edges, start, stop)
    path = get_path(dijkstra_path)
    
    # Create the response, which is a list of points, each with x and y
    response = {
        "points": [{"x": x, "y": y} for x,y in path]
    }
    
    # Return the response, along with True, indicating that the module
    # should keep running.
    return (True, response)

Send handler function to the run function in the `laps.py` module, which runs the function and checks for any job request to the module. If there is any requests, it pops it off the list, and calls the handler function with the `Job request` in the JSON form above.

(As of now, the hardcoded job request that is recieved from backend is: `{'job_id': 20, 'start': {'x': 200.0, 'y': 90.0}, 'stop': {'x': 10.0, 'y': 10.0}, 'map_id': 2}`)

In [ ]:
with laps.Runner("dijkstra_module", "1.0.0", redis_connection) as runner:
    runner.run(handler)